In [41]:
import pandas as pd

In [42]:
df=pd.read_csv(r"D:\Downloads\Telecom_customer churn.csv")

In [43]:
df1=df.copy();
#getting 80% of the data for the training set.
training=df1.sample(frac=.8)
#just reviewing the effect of using the sample function on the dataframe
training

,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,change_mou,...,forgntvl,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd,eqpdays,Customer_ID
71870,50.0250,43.00,53.5075,0.0000,0.00,0.0000,0.0000,0.0,0.0000,-43.00,...,0.0,S,U,U,U,U,U,N,246.0,1071871
95636,46.8475,603.25,10.0000,0.2475,97.25,38.9000,38.9000,0.0,0.0000,-354.25,...,0.0,P,U,U,U,U,U,N,181.0,1095637
43547,156.7050,1762.50,44.9900,0.9900,333.25,116.6375,116.6375,0.0,3.1250,109.50,...,0.0,Z,U,U,U,U,U,N,199.0,1043548
75320,86.4900,745.00,92.4900,0.0000,0.00,0.0000,0.0000,0.0,0.0000,-105.00,...,0.0,F,U,U,U,U,U,Y,290.0,1075321
74883,19.3925,469.25,42.3250,0.4950,10.00,3.9000,0.0000,3.9,0.0000,-0.25,...,0.0,N,U,U,U,U,U,N,146.0,1074884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92734,38.5900,535.00,44.9900,0.0000,9.00,3.6000,3.6000,0.0,0.0000,4.00,...,0.0,N,U,U,U,U,U,Y,155.0,1092735
12804,36.0775,59.25,29.9900,0.9900,0.00,0.0000,0.0000,0.0,5.0975,10.75,...,1.0,G,U,U,U,U,U,Y,859.0,1012805
91235,61.1500,88.25,69.9900,0.0000,0.00,0.0000,0.0000,0.0,0.5850,-12.25,...,1.0,B,U,Y,U,U,U,Y,250.0,1091236
58065,63.1175,807.00,49.9900,1.2375,27.25,8.1750,8.1750,0.0,3.7150,93.00,...,0.0,G,U,U,Y,U,U,Y,391.0,1058066


In [95]:
corr1 = df.copy()

Correlation = corr1.corr()['churn']
print(Correlation)

rev_Mean      -0.010883
mou_Mean      -0.057027
totmrc_Mean   -0.068558
da_Mean       -0.013551
ovrmou_Mean    0.019720
                 ...   
income         0.005260
numbcars       0.002392
forgntvl      -0.004986
eqpdays        0.112691
Customer_ID   -0.042764
Name: churn, Length: 79, dtype: float64


In [61]:
#getting 20% of the data for the test set.
test=df1.drop(training.index);
test
test_num = test.select_dtypes(exclude=["object_","bool_"])

for column in test_num :
        test_num[column].fillna(test_num[column].min()-0.5,inplace=True);

test_cat = test.select_dtypes(exclude=["number","float_"])

for column in test_cat :
    test_cat[column].fillna("null",inplace=True)

C:\Users\farou\anaconda3\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [45]:
#getting the numerical values for the training dataset  
training_num=training.select_dtypes(exclude=["object_","bool_"]);


In [46]:
for column in training_num :
        training_num[column].fillna(training_num[column].min()-0.5,inplace=True);

In [47]:
training_num.isnull().sum()

rev_Mean       0
mou_Mean       0
totmrc_Mean    0
da_Mean        0
ovrmou_Mean    0
              ..
income         0
numbcars       0
forgntvl       0
eqpdays        0
Customer_ID    0
Length: 79, dtype: int64

In [48]:
%matplotlib inline

import warnings
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm
from scipy.stats._continuous_distns import _distn_names
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
matplotlib.style.use('ggplot')

# Create models from data
def best_fit_distribution(data, bins=200, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    # Best holders
    best_distributions = []

    # Estimate distribution parameters from data
    for ii, distribution in enumerate([d for d in _distn_names if not d in ['levy_stable', 'studentized_range','gumbel_r','gumbel_l','kstwo','burr12','gompertz','invgamma','norminvgauss','maxwell','kappa3','ncf','t','pareto','powernorm','semicircular','uniform']]):

        print("{:>3} / {:<3}: {}".format( ii+1, len(_distn_names), distribution ))

        distribution = getattr(st, distribution)

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')
                
                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]
                
                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))
                
                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                    end
                except Exception:
                    pass

                # identify if this distribution is better
                best_distributions.append((distribution, params, sse))
        
        except Exception:
            pass

    
    return sorted(best_distributions, key=lambda x:x[2])



In [49]:
import numpy as np
def get_pdf_prob(a, dist, params, size=10000):
    """Generate distributions's Probability Distribution Function """

    # Separate parts of parameters
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]

    # Get sane start and end points of distribution
    start = dist.ppf(0.01, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.01, loc=loc, scale=scale)
    end = dist.ppf(0.99, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.99, loc=loc, scale=scale)

    # Build PDF and turn into pandas Series
    x = np.linspace(start, end, size)
    y = dist.pdf(x, loc=loc, scale=scale, *arg) 
    pdf = pd.Series(y, x)
    arrx = pdf.index
    ##### we will find the closest number to the input (the value of the column of this client)
    absolute_val_array = np.abs(arrx - a)
    smallest_difference_index = absolute_val_array.argmin()
    closest_element = arrx[smallest_difference_index]
    
    index = np.where(arrx == closest_element)
    yval = y[index[0][0]]
    
    return  yval


In [50]:
def get_pmf_prob(find,clientNumber,j):
    column = training_cat.iloc[clientNumber].index[j]
    pmf = training_cat[column].value_counts().sort_index() / len(training_cat)
    arrx = pmf.index
    index = np.where(arrx == find) # find is the string to find
    yval = pmf.iloc[index[0][0]]
    
    return yval
 

In [60]:
def get_pmf_prob_churn(find,clientNumber,j):
    column = churn_cat.iloc[clientNumber].index[j]
    pmf = churn_cat[column].value_counts().sort_index() / len(churn_cat)
    arrx = pmf.index
    index = np.where(arrx == find) # find is the string to find
    yval = pmf.iloc[index[0][0]]
    
    return yval

In [51]:
churn_d = training.copy()
churn_d.drop(churn_d[churn_d.churn == 0].index, inplace = True)
prChurn = len(churn_d)/len(training)
print("prChurn = ",+ prChurn)

prChurn =  0.496925


In [52]:
# FINDING MISSING CAT DATA AND SUBSTITUTING THE NULLS WITH NULL AND CREATING A NEW CATEGORY  

training_cat = training.select_dtypes(exclude=["number","float_"])

for column in training_cat :
    training_cat[column].fillna("null",inplace=True)   

C:\Users\farou\anaconda3\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [53]:
churn_num_data=churn_d.select_dtypes(exclude=["object_","bool_"])

for column in churn_num_data :
        churn_num_data[column].fillna(churn_num_data[column].min()-0.5,inplace=True);

In [54]:
churn_cat = churn_d.select_dtypes(exclude=["number","float_"])

for column in churn_cat :
    churn_cat[column].fillna("null",inplace=True)

In [97]:
# numerator pdf

churnlength = len(churn_num_data)
churndist = []
churnparams = []


#for i in range (0, churnlength):
# for i in range(0, 4):
for i in ['change_mou','peak_vce_Mean','mou_peav_Mean','mou_Mean']:
    data = pd.Series(churn_num_data[i])


    # # Find best fit distribution
    best_distibutions = best_fit_distribution(data, 200 , ax);
    best_dist = best_distibutions[0]
    
    churndist.append(best_dist[0])
    churnparams.append(best_dist[1])
    print(churndist)
    print(churnparams)
    
    print("Column is done")

  1 / 104: ksone
  2 / 104: kstwobign
  3 / 104: norm
  4 / 104: alpha
  5 / 104: anglit
  6 / 104: arcsine
  7 / 104: beta
  8 / 104: betaprime
  9 / 104: bradford
 10 / 104: burr
 11 / 104: fisk
 12 / 104: cauchy
 13 / 104: chi
 14 / 104: chi2
 15 / 104: cosine
 16 / 104: dgamma
 17 / 104: dweibull
 18 / 104: expon
 19 / 104: exponnorm
 20 / 104: exponweib
 21 / 104: exponpow
 22 / 104: fatiguelife
 23 / 104: foldcauchy
 24 / 104: f
 25 / 104: foldnorm
 26 / 104: weibull_min
 27 / 104: weibull_max
 28 / 104: genlogistic
 29 / 104: genpareto
 30 / 104: genexpon
 31 / 104: genextreme
 32 / 104: gamma
 33 / 104: erlang
 34 / 104: gengamma
 35 / 104: genhalflogistic
 36 / 104: genhyperbolic
 37 / 104: halfcauchy
 38 / 104: halflogistic
 39 / 104: halfnorm
 40 / 104: hypsecant
 41 / 104: gausshyper
 42 / 104: invgauss
 43 / 104: geninvgauss
 44 / 104: invweibull
 45 / 104: johnsonsb
 46 / 104: johnsonsu
 47 / 104: laplace
 48 / 104: laplace_asymmetric
 49 / 104: levy
 50 / 104: levy_l
 51

In [98]:
# denominator pdf

generallength = len(churn_num_data)
generaldist = []
generalparams = []


#for i in range (0, generallength):
for i in ['change_mou','peak_vce_Mean','mou_peav_Mean','mou_Mean']:
    data =pd.Series(training_num[i])

    # # Find best fit distribution
    best_distibutions = best_fit_distribution(data, 200 , ax);
    best_dist = best_distibutions[0]
    
    generaldist.append(best_dist[0])
    generalparams.append(best_dist[1])
    print(generaldist)
    print(generalparams)
    
    print("Column is done")

  1 / 104: ksone
  2 / 104: kstwobign
  3 / 104: norm
  4 / 104: alpha
  5 / 104: anglit
  6 / 104: arcsine
  7 / 104: beta
  8 / 104: betaprime
  9 / 104: bradford
 10 / 104: burr
 11 / 104: fisk
 12 / 104: cauchy
 13 / 104: chi
 14 / 104: chi2
 15 / 104: cosine
 16 / 104: dgamma
 17 / 104: dweibull
 18 / 104: expon
 19 / 104: exponnorm
 20 / 104: exponweib
 21 / 104: exponpow
 22 / 104: fatiguelife
 23 / 104: foldcauchy
 24 / 104: f
 25 / 104: foldnorm
 26 / 104: weibull_min
 27 / 104: weibull_max
 28 / 104: genlogistic
 29 / 104: genpareto
 30 / 104: genexpon
 31 / 104: genextreme
 32 / 104: gamma
 33 / 104: erlang
 34 / 104: gengamma
 35 / 104: genhalflogistic
 36 / 104: genhyperbolic
 37 / 104: halfcauchy
 38 / 104: halflogistic
 39 / 104: halfnorm
 40 / 104: hypsecant
 41 / 104: gausshyper
 42 / 104: invgauss
 43 / 104: geninvgauss
 44 / 104: invweibull
 45 / 104: johnsonsb
 46 / 104: johnsonsu
 47 / 104: laplace
 48 / 104: laplace_asymmetric
 49 / 104: levy
 50 / 104: levy_l
 51

In [99]:
def willChurn(c_num,c_cat,cNumber): 
    
    
    #numerator given churn
    
    churnResult = 1
    
    # pdf multiplication given churn
#    for i in ['change_mou','peak_vce_Mean','mou_peav_Mean','mou_Mean']:
#     for i in [1,9,35,37]:
#         cpar = c_num[i]
#         generalResult = generalResult * get_pdf_prob(cpar, churndist[i], churnparams[i])
    
    for i in [1,9,35,37]:
        cpar = c_num[i]
        if i == 1:
            churnResult = churnResult * get_pdf_prob(cpar, churndist[0], churnparams[0])
        if i == 9:
            churnResult = churnResult * get_pdf_prob(cpar, churndist[1], churnparams[1])
        if i == 35:
            churnResult = churnResult * get_pdf_prob(cpar, churndist[2], churnparams[2])
        if i == 37:
            churnResult = churnResult * get_pdf_prob(cpar, churndist[3], churnparams[3])
    
    # pmf multiplication given churn
    
    for j in range(0,4):
        cpar = c_cat[j]
        churnResult = churnResult * get_pmf_prob_churn(cpar, cNumber, j)
    
    
    
    ##############################################
    
    #denominator
    
    generalResult = 1
    
    # pdf multiplication general
    
#    for i in ['change_mou','peak_vce_Mean','mou_peav_Mean','mou_Mean']:
#     for i in [1,9,35,37]:
#         cpar = c_num[i]
#         generalResult = generalResult * get_pdf_prob(cpar, generaldist[i], generalparams[i])
    
    for i in [1,9,35,37]:
        cpar = c_num[i]
        if i == 1:
            churnResult = churnResult * get_pdf_prob(cpar, generaldist[0], generalparams[0])
        if i == 9:
            churnResult = churnResult * get_pdf_prob(cpar, generaldist[1], generalparams[1])
        if i == 35:
            churnResult = churnResult * get_pdf_prob(cpar, generaldist[2], generalparams[2])
        if i == 37:
            churnResult = churnResult * get_pdf_prob(cpar, generaldist[3], generalparams[3])
    
    # pmf multiplication general
    
    for j in range(0,4):
        cpar = c_cat[j]
        churnResult = churnResult * get_pmf_prob(cpar, cNumber, j)
        
    ##############################################
    
    prchurnrow = (churnResult/generalResult)*prChurn
    
    print("Client will churn: " + str(prchurnrow*100) + " %")
    
    return prchurnrow*100

In [100]:
willChurn(test_num.iloc[2005],test_cat.iloc[2005], 2005)

Client will churn: 5.694989417428802e-20 %


5.694989417428802e-20

In [101]:
print(test_num.iloc[2005][48])

1.0


In [102]:
print(test.iloc[134])

rev_Mean       276.1525
mou_Mean         1680.0
totmrc_Mean      100.87
da_Mean          0.2475
ovrmou_Mean       699.5
                 ...   
kid11_15              U
kid16_17              U
creditcd              Y
eqpdays           630.0
Customer_ID     1000601
Name: 600, Length: 100, dtype: object


In [88]:
import math

testlen = len(test_num)
correctCounter = 0
#for i in range(0,testlen):
for i in range(0, 2000):
    check = willChurn(test_num.iloc[i],test_cat.iloc[i],i)
    if check >= 0.5:
        churndecision = 1.0
    else: 
        churndecision = 0.0
    
    if int(churndecision) == int(test_num.iloc[i][48]):
        correctCounter = correctCounter + 1
    
    print(i)    

accuracy = correctCounter/2000
print(str(math.ceil(accuracy*100)) + "%")
        

Client will churn: 0.00012994753353978177 %
0
Client will churn: 0.0019438887210381495 %
1
Client will churn: 0.004043659988351732 %
2
Client will churn: 0.0001303528529727245 %
3
Client will churn: 0.0003775895607117476 %
4
Client will churn: 0.0047909348119455995 %
5
Client will churn: 0.005301856462114128 %
6
Client will churn: 6.185454304027904e-05 %
7
Client will churn: 3.721502606381697e-05 %
8
Client will churn: 0.0017248357385325646 %
9
Client will churn: 0.0007084732782720814 %
10
Client will churn: 1.0134319625637357e-05 %
11
Client will churn: 0.0002075261137453043 %
12
Client will churn: 0.0019098503304069824 %
13
Client will churn: 0.00012453517887182214 %
14
Client will churn: 1.1194128528419432e-05 %
15
Client will churn: 6.607345191107872e-05 %
16
Client will churn: 1.8970114936317597e-05 %
17
Client will churn: 0.00012253895510391832 %
18
Client will churn: 4.619987953403299e-05 %
19
Client will churn: 0.0030750756403629662 %
20
Client will churn: 0.0006912307318432874

In [178]:
import math
print(str(math.ceil(accuracy*100)) + "%")

58 %


In [22]:
print(test_num.iloc[134].index[2])

totmrc_Mean


In [25]:
for column in training_cat:
    pmf = training_cat[column].value_counts().sort_index() / len(training_cat)
    print(pmf)
    print()

N    0.137425
U    0.669462
Y    0.193112
Name: new_cell, dtype: float64
A     0.169437
A2    0.008700
A3    0.000025
AA    0.364737
B     0.040813
B2    0.001138
BA    0.127188
C     0.014663
C2    0.001888
C5    0.000775
CA    0.083175
CC    0.000262
CY    0.001863
D     0.002525
D2    0.000063
D4    0.003012
D5    0.001250
DA    0.036550
E     0.003537
E2    0.000225
E4    0.008313
EA    0.064037
EC    0.000525
EF    0.000025
EM    0.000525
G     0.002500
GA    0.002750
GY    0.000350
H     0.000063
I     0.002400
IF    0.000138
J     0.001962
JF    0.001975
K     0.001175
L     0.000050
M     0.001825
O     0.000250
P1    0.000050
S     0.000013
TP    0.000050
U     0.004700
U1    0.000612
V1    0.000725
W     0.000963
Y     0.000550
Z     0.001900
Z1    0.000112
Z2    0.000050
Z4    0.002437
Z5    0.000562
ZA    0.035412
ZF    0.000013
ZY    0.001162
Name: crclscod, dtype: float64
N    0.8605
Y    0.1395
Name: asl_flag, dtype: float64
C       0.169925
R       0.049200
S       0.32

In [37]:
column = training_cat.iloc[134].index[5]
pmf = training_cat[column].value_counts().sort_index() / len(training_cat)
print(pmf)
arrx = pmf.index
print(arrx)
index = np.where(arrx == 'T')
print(index[0][0])
yval = pmf.iloc[index[0][0]]
print(yval)

N       0.232475
T       0.042963
U       0.002275
Y       0.722275
null    0.000013
Name: dualband, dtype: float64
Index(['N', 'T', 'U', 'Y', 'null'], dtype='object')
1
0.0429625
